# Liiketoiminnan tunnusluvuista

Tämä kirjanen esittelee hieman miten pörssiyhtiöiden tietoja pystyy tarkastelemaan ohjelmoinnin avulla.

### 1. Työkalut

In [ ]:
# Tuodaan tarvittavat kirjastot

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Jotkut Pythonin versiot eivät oletusarvoisesti tue pandaksen kautta Excel-tiedostojen lukemista. Alla oleva käsky auttaa virheilmoituksiin.
%pip install openpyxl

### 2. Aineistot

In [ ]:
# Haetaan tarkasteltavia tietoja.
# Tässä on käytetty Kauppalehden listaa Helsingin pörssistä, jonka on tuottanut Alma Talent Tietopalvelut.
# Esimerkkiaineisto on haettu 21.4.2025.
# https://www.kauppalehti.fi/porssi/indeksit/omxhpi

data = pd.read_excel("https://github.com/opendata-education/Tyopajat/raw/refs/heads/main/materiaali/data/AFAGR_financialstmt.xlsx", header=1)

In [ ]:
# Vilkaistaan aineistoon.
data

In [ ]:
# Huomataan, että tiedoston lopussa on turhia osasia. Poistetaan ne rivi-indeksien avulla.
data = data.drop([40, 41, 42, 43, 44])

# Siistitään samalla nimisaraketta hieman
data = data.rename(columns = {"Afarak Group Oyj": "Afarak_Group_Oyj"})

In [ ]:
# Tarkastetaan, että tulos näyttää oikealta.
data

### 3. Analyysi

Lähdetään tarkastelemaan ensin yhden muuttujan kohdalla miten rivistä saadaan yksinkertainen kuvaaja ulos ennen kuin alamme laskemaan niistä tarkempia lukuja.

In [ ]:
# Määritetään mitä riviä halutaan tarkastella.
ehto = "Nettovelkaantumisaste, %"

# Kyseisen rivin aineistot halutaan y-akselille, joten tehdään niistä lista.
pysty = data.query('Afarak_Group_Oyj == @ehto')
pysty = pysty.iloc[0][1:].tolist()

# Käytetään vaaka-akselina sarakeotsikoiden päiväyksiä.
vaaka = data.columns[1:].tolist()

# Piirretään kuvaajat.
plt.figure(figsize = (10,5))
plt.plot(vaaka, pysty, c = "y")
plt.scatter(vaaka, pysty, c = "firebrick")

# Koristellaan kuvaaja soveltuvilla selitteillä.
plt.title(ehto)
plt.xticks(rotation = 30)
plt.show()

Tyypillisesti meitä kiinnostaa erilaisten keskiarvojen tai poikkeamien laskeskelu. Yksi tapa tehdä näin on käyttää numpy-kirjaston mean()-käskyä, joka soveltuu lista-tyyppisten muuttujien käsittelyyn. 

In [ ]:
print(f"Keskiarvo muuttujalle {ehto} on : " + str(np.mean(pysty)))

Voisimme tietysti yhdistää useamman eri muuttujan keskiarvoineen samaan kuvaajaan helpottamaan vertailutyötämme.

In [ ]:
# Määritetään mitä riviä halutaan tarkastella.
ehto1 = "P/E-luku"
ehto2 = "EV/EBIT-luku"

# Kyseisten rivien aineistot halutaan y-akselille, joten tehdään niistä listat.
y1 = data.query('Afarak_Group_Oyj == @ehto1')
y1 = y1.iloc[0][1:].tolist()

y2 = data.query('Afarak_Group_Oyj == @ehto2')
y2 = y2.iloc[0][1:].tolist()

# Käytetään vaaka-akselina sarakeotsikoiden päiväyksiä.
vaaka = data.columns[1:].tolist()

# Lasketaan kummankin muuttujan keskiarvot.
KA1 = np.mean(y1)
KA2 = np.mean(y2)

# Piirretään kuvaajat.
plt.figure(figsize = (10,5))

plt.plot(vaaka, y1, c = "y")
plt.scatter(vaaka, y1, c = "firebrick", label = f"{ehto1}, keskiarvo {str(KA1)}")

plt.plot(vaaka, y2, c = "slateblue")
plt.scatter(vaaka, y2, c = "navy", label = f"{ehto2}, keskiarvo {str(KA2)}")

# Piirretään vielä keskiarvotkin mukaan.
plt.axhline(y = KA1, color = 'wheat', linestyle = 'dashed')
plt.axhline(y = KA2, color = 'lightsteelblue', linestyle = 'dashed')

# Koristellaan kuvaaja soveltuvilla selitteillä.
plt.title(f"Vertailussa {ehto1} ja {ehto2} \n", fontsize = 20)
plt.xticks(rotation = 30)
plt.legend()
plt.show()

Ylläolevasta kuvasta voi miettiä miten hyvä työkalu koko kymmenvuotisen kauden keskiarvo on kuvaamaan yrityksen yleistä tilaa tai muutosta. Tähän tarkoitukseen voisi käyttää jotakin sopivampaa, kuten trendiviivaa.

In [ ]:
# Sovitetaan äskeisiin havaintoihin yleiskuvaa tasoittava trendiviiva.
plt.figure(figsize = (10, 5))

# Aineisto y2.
plt.scatter(vaaka, y2, c = "navy", label = f"{ehto2}, keskiarvo {str(KA2)}")

# Trendi.
x = np.arange(len(vaaka))
y = y2.copy()

# Tästä lukua vaihtamalla voi muuttaa sovitetun yhtälön astetta.
z = np.polyfit(x, y, 3)
p = np.poly1d(z)
plt.plot(vaaka, p(x), c = "black", linestyle = "dashed", label = f"{ehto2}", alpha = 0.7)

# Selitteet.
plt.legend(loc = "lower left")
plt.title(f"{ehto2} \n", fontsize = 20)
plt.xticks(rotation = 30)
plt.show()


### Ideoita omiin testailuihin

Kauppalehden sivuilta voit etsiä eri yrityksiä ja yhtiöitä oman mielenkiintosi mukaan.
Niiden aineistoista voisi kysyä esimerkiksi seuraavia:

- Liikevaihdon ja tuloksen keskimääräinen kasvuprosentti viideltä ja kymmeneltä vuodelta.
- Viiden vuoden ja 10 vuoden keskiarvo P/E-luvulle sekä EV/EBIT-luvulle, niiden vertaaminen nykyisiin P/E- ja EV/EBIT-lukuihin.
- Oman pääoman tuottoprosentin suhde P/B-lukuun sekä näistä viiden ja 10 vuoden keskiarvo.
- Sijoitetun pääoman tuotto, viiden ja 10 vuoden keskiarvo.
- Nettovelkaantumisaste yleisesti.